In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
properties_518d723fd6514bbf8b04bcadda4fba64 = {
    'driver': 'com.ibm.db2.jcc.DB2Driver',
    'jdbcurl': 'jdbc:db2://169.46.39.90:50000/BLUDB',
    'user': 'equipo14',
    'password': 'TEAM14#IBM!Db2'
}
catalogo_total = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.CATALOGO_TOTAL', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
centros = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.CENTROS', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
company_total = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.COMPANY_TOTAL', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
item = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.ITEM', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
pagos = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.PAGOS', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
personas = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.PERSONA', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
retail = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.RETAIL', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
unicard = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.UNICARD', properties=properties_518d723fd6514bbf8b04bcadda4fba64)
unired_total = spark.read.jdbc(properties_518d723fd6514bbf8b04bcadda4fba64['jdbcurl'], table='BLUADMIN.UNIRED_TOTAL', properties=properties_518d723fd6514bbf8b04bcadda4fba64)

## Procesamiento pagos Unired

In [3]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import (dayofmonth, hour, month, year, weekofyear,format_number,date_format)

unicard_limpia = unicard.select(['CUSTOMER_ID',month('FECHA_ACTIVIDAD').alias('mes'),year('FECHA_ACTIVIDAD').alias('ano'),dayofmonth('FECHA_ACTIVIDAD').alias('dia'),'BLOQUEO'])
unicard_limpia.orderBy([unicard_limpia['CUSTOMER_ID'].desc(),unicard_limpia['ano'].desc(),unicard_limpia['mes'].desc(),unicard_limpia['dia'].desc()]).show()

#https://stackoverflow.com/questions/38397796/retrieve-top-n-in-each-group-of-a-dataframe-in-pyspark
#unicard_ordenada = unicard_limpia.orderBy(['CUSTOMER_ID','ano','mes'])
window = Window.partitionBy(unicard_limpia['CUSTOMER_ID']).orderBy([unicard_limpia['CUSTOMER_ID'].desc(),unicard_limpia['ano'].desc(),unicard_limpia['mes'].desc(),unicard_limpia['dia'].desc()])

#.show()
unicard_limpia = unicard_limpia.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 1).orderBy(unicard_limpia['CUSTOMER_ID'].desc()).select(['CUSTOMER_ID','bloqueo'])
unicard_limpia.show()

+-----------+---+----+---+-------+
|CUSTOMER_ID|mes| ano|dia|BLOQUEO|
+-----------+---+----+---+-------+
|   99996370|  9|2017|  5|      0|
|   99996370|  8|2017|  7|      0|
|   99996370|  7|2017| 31|      0|
|   99996370|  6|2017| 30|      0|
|   99996370|  5|2017| 31|      0|
|   99996370|  4|2017| 30|      0|
|   99996370|  3|2017| 31|      0|
|   99996370|  2|2017| 28|      0|
|   99996370|  1|2017| 31|      0|
|   99995540|  4|2017| 30|      1|
|   99995540|  3|2017| 31|      1|
|   99995540|  2|2017| 28|      1|
|   99995540|  1|2017| 31|      1|
|   99993156|  9|2017|  5|      1|
|   99993156|  8|2017|  7|      0|
|   99993156|  7|2017| 31|      0|
|   99993156|  6|2017| 30|      0|
|   99993156|  5|2017| 31|      0|
|   99993156|  4|2017| 30|      0|
|   99993156|  3|2017| 31|      0|
+-----------+---+----+---+-------+
only showing top 20 rows

+-----------+-------+
|CUSTOMER_ID|bloqueo|
+-----------+-------+
|   99996370|      0|
|   99995540|      1|
|   99993156|      1|
| 

In [4]:
#Se va a realizar el join con las personas que poseen o no la tarjeta unimarc bloqueada
selectPersonas = ['CUSTOMER_ID',
 'GENDER',
 'EDAD',
 'MOBILE_CONTACTABILITY',
 'EMAIL_CONTACTABILITY',
 'GSE',
 'CIVIL_STATUS']

personas_tarjeta = personas.select(selectPersonas).join(unicard_limpia,"CUSTOMER_ID")
personas_tarjeta.orderBy(personas_tarjeta['CUSTOMER_ID'].desc()).show()

+-----------+---------+----+---------------------+--------------------+---+------------+-------+
|CUSTOMER_ID|   GENDER|EDAD|MOBILE_CONTACTABILITY|EMAIL_CONTACTABILITY|GSE|CIVIL_STATUS|bloqueo|
+-----------+---------+----+---------------------+--------------------+---+------------+-------+
|   99996370|MASCULINO|  58|                    1|                   1|  D|      CASADO|      0|
|   99995540| FEMENINO|  60|                    1|                   0|  D|     SOLTERO|      1|
|   99993156| FEMENINO|  59|                    1|                   1|  E|      CASADO|      1|
|   99987692| FEMENINO|  57|                    0|                   0| C3|      CASADO|      0|
|   99985778| FEMENINO|  70|                    1|                   0| C3|  DIVORCIADO|      1|
|   99978096| FEMENINO|  61|                    1|                   0|  E|     SOLTERO|      0|
|   99974474|MASCULINO|  63|                    1|                   1| C3|       VIUDO|      1|
|   99974456| FEMENINO|  65|  

In [5]:
#Agrupamos los pagos mensuales de unired por usuario
pagos_unired = unired_total.select(['CUSTOMER_ID',month('HED_FECHATRX').alias('mes_unired'),year('HED_FECHATRX').alias('ano_unired'),'SPEND_AMT'])
pagos_unired = pagos_unired.groupBy(['CUSTOMER_ID','ano_unired','mes_unired']).agg({'SPEND_AMT':'sum'}).withColumnRenamed("sum(SPEND_AMT)", "pago_unired").orderBy(['CUSTOMER_ID','ano_unired','mes_unired'])
pagos_unired.show()

+-----------+----------+----------+-----------+
|CUSTOMER_ID|ano_unired|mes_unired|pago_unired|
+-----------+----------+----------+-----------+
|        101|      2017|         8|      57780|
|        166|      2016|         9|      10274|
|        166|      2016|        10|      10307|
|        166|      2016|        11|      12713|
|        166|      2016|        12|      40094|
|        166|      2017|         2|      16291|
|        166|      2017|         3|      30434|
|        166|      2017|         4|      29559|
|        166|      2017|         5|      20101|
|        166|      2017|         6|      18942|
|        192|      2017|         7|      37716|
|        839|      2017|         6|      36925|
|        839|      2017|         8|      27309|
|        839|      2017|         9|      27309|
|        841|      2016|         9|      44020|
|        841|      2016|        10|      35971|
|        841|      2016|        11|     126210|
|       1055|      2017|         3|     

In [6]:
## Juntamos la información de las personas con el pago de unired
personas_unired = personas_tarjeta.join(pagos_unired,"CUSTOMER_ID")
personas_unired.orderBy(personas_unired['CUSTOMER_ID'].desc()).show()

+-----------+---------+----+---------------------+--------------------+---+------------+-------+----------+----------+-----------+
|CUSTOMER_ID|   GENDER|EDAD|MOBILE_CONTACTABILITY|EMAIL_CONTACTABILITY|GSE|CIVIL_STATUS|bloqueo|ano_unired|mes_unired|pago_unired|
+-----------+---------+----+---------------------+--------------------+---+------------+-------+----------+----------+-----------+
|   99974456| FEMENINO|  65|                    1|                   0|  E|      CASADO|      0|      2017|         6|     171678|
|   99974456| FEMENINO|  65|                    1|                   0|  E|      CASADO|      0|      2016|        10|     150462|
|   99974456| FEMENINO|  65|                    1|                   0|  E|      CASADO|      0|      2016|         9|     114343|
|   99919770|MASCULINO|  39|                    1|                   1| C3|     SOLTERO|      0|      2017|         6|      51600|
|   99919770|MASCULINO|  39|                    1|                   1| C3|     SOL

In [7]:
from pyspark.sql.window import *
from pyspark.sql.functions import row_number

resp = personas_unired.withColumn("row_num", row_number().over(Window.partitionBy("CUSTOMER_ID").orderBy(['CUSTOMER_ID'])))
resp.orderBy(resp["row_num"].desc()).show()

+-----------+---------+----+---------------------+--------------------+----+------------+-------+----------+----------+-----------+-------+
|CUSTOMER_ID|   GENDER|EDAD|MOBILE_CONTACTABILITY|EMAIL_CONTACTABILITY| GSE|CIVIL_STATUS|bloqueo|ano_unired|mes_unired|pago_unired|row_num|
+-----------+---------+----+---------------------+--------------------+----+------------+-------+----------+----------+-----------+-------+
|   18164436| FEMENINO|  50|                    1|                   0|   D|     SOLTERO|      0|      2017|         9|     124300|     13|
|   29489104|MASCULINO|  60|                    1|                   0|ABC1|      CASADO|      0|      2017|         9|      49570|     13|
|   74721344|MASCULINO|  50|                    1|                   0|  C3|      CASADO|      0|      2017|         9|      73880|     13|
|   74522506|MASCULINO|  52|                    1|                   0|  C3|      CASADO|      0|      2017|         9|     133150|     13|
|   12950772| FEMENI

In [ ]:
## Separamos a los que están bloqueados de quienes no lo están
resp.createOrReplaceTempView("resp")
sbloq = spark.sql("SELECT * FROM resp WHERE bloqueo=1")
nbloq = spark.sql("SELECT * FROM resp WHERE bloqueo=0")

#from pyspark.sql.functions import size, collect_list
#sbloq.groupBy('row_num').agg(size(collect_list('row_num'))).show()
#import matplotlib.pyplot as plt
#import numpy as np

#data = Data([Histogram(x=sbloq.toPandas()['row_num'])])
#py.iplot(data)
#plt.hist(sbloq.toPandas()['row_num'], bins = 20)
#plt.xlabel("Mes")
#plt.ylabel("Cantidad")
#plt.title("Cantidad de meses retail bloqueados")
#plt.show()

#plt.hist(nbloq.toPandas()['row_num'], bins = 20)
#plt.xlabel("Mes")
#plt.ylabel("Cantidad")
#plt.title("Cantidad de meses retail no bloqueados")
#plt.show()
#data = Data([Histogram(x=nbloq.toPandas()['row_num'])])

#nbloq.groupBy('row_num').agg(size(collect_list('row_num'))).show()

In [8]:
aux1 = resp.select(['CUSTOMER_ID', 'bloqueo', 'ano_unired', 'mes_unired', 'pago_unired', 'row_num'])

In [9]:
from pyspark.sql import functions as F

personas_unired2 = aux1.groupBy(['CUSTOMER_ID', 'bloqueo']).agg(F.stddev('pago_unired'), F.max('pago_unired'), F.avg('pago_unired')).withColumnRenamed("stddev_samp(pago_unired)", "std_pago_unired").withColumnRenamed("max(pago_unired)", "max_pago_unired").withColumnRenamed("avg(pago_unired)", "avg_pago_unired" ).na.drop(how="any")
personas_unired2.show()

personas_unired3 = aux1.groupBy(['CUSTOMER_ID', 'bloqueo']).agg({'pago_unired':'sum'}).withColumnRenamed("sum(pago_unired)", "pago_unired_mensual")
personas_unired3.show()



+-----------+-------+------------------+---------------+------------------+
|CUSTOMER_ID|bloqueo|   std_pago_unired|max_pago_unired|   avg_pago_unired|
+-----------+-------+------------------+---------------+------------------+
|    9097662|      1| 8457.780300944665|          28450| 11578.57142857143|
|   12950772|      0| 45061.98224639434|         238720|138554.84615384616|
|   16303304|      0| 29717.87208622672|          78250|44130.666666666664|
|   29489104|      0|16493.257609608936|          72224| 50852.07692307692|
|   56198512|      0|25356.571422132653|          89850|           47650.0|
|    5696398|      0|11337.350048622473|          42379|         14926.875|
|   34225754|      0| 37175.86283514989|         155749| 86922.91666666667|
|   40482792|      0|25782.624941248927|          92819| 46018.88888888889|
|   42861006|      0| 8164.721585665281|          36500|           21662.5|
|   58218984|      0| 11828.25551683039|          53910|           37025.0|
|   62114692

In [13]:
pagos_uni = personas_unired2.join(personas_unired3,'CUSTOMER_ID')
pagos_uni = pagos_uni.drop('BLOQUEO')

## Procesamiento fechas de pago Unired

In [14]:
company    = company_total.select(['COMPANY_ID', 'INDUSTRY'])
unired_aux = unired_total.select(['CUSTOMER_ID',dayofmonth('HED_FECHATRX').alias('dia_unired'),'COMPANY_ID'])
unired_aux.printSchema()

root
 |-- CUSTOMER_ID: integer (nullable = true)
 |-- dia_unired: integer (nullable = true)
 |-- COMPANY_ID: integer (nullable = true)



In [15]:
aux1 = unired_aux

unired1 = aux1.groupBy(['CUSTOMER_ID','COMPANY_ID']).agg(F.stddev('dia_unired')).withColumnRenamed("stddev_samp(dia_unired)", "std_dia_pago_unired").na.drop(how="any")
unired1 = unired1.groupBy(['CUSTOMER_ID']).agg(F.avg('std_dia_pago_unired'), F.max('std_dia_pago_unired'), F.min('std_dia_pago_unired')).withColumnRenamed("avg(std_dia_pago_unired)", "avg_dia_pago_unired").withColumnRenamed("max(std_dia_pago_unired)", "max_dia_pago_unired").withColumnRenamed("min(std_dia_pago_unired)", "min_dia_pago_unired" ).na.drop(how="any")

## Procesamiento pagos en el Retail

In [16]:
pagos_distintos = pagos.select(['PAYMENT_MEAN_ID','PAYMENT_MEAN_GROUP_NAME']).distinct()

In [66]:
retail.printSchema()

root
 |-- CUSTOMER_ID: integer (nullable = true)
 |-- ORGANIZATION_ID: integer (nullable = true)
 |-- ITEM_ARTICLE_ID: integer (nullable = true)
 |-- CENTER_ID: integer (nullable = true)
 |-- HED_LOCAL: integer (nullable = true)
 |-- HED_FECHATRX: timestamp (nullable = true)
 |-- HED_HORATRX: timestamp (nullable = true)
 |-- HED_NUMTRX: integer (nullable = true)
 |-- HED_POS: integer (nullable = true)
 |-- HED_TIPODOC: string (nullable = true)
 |-- ITEM_QTY_UMB: decimal(28,12) (nullable = true)
 |-- DISCOUNT_AMT: integer (nullable = true)
 |-- SPEND_AMT: integer (nullable = true)
 |-- SERVICE_SPEND_AMT: integer (nullable = true)
 |-- PAYMENT_MEAN_ID: integer (nullable = true)
 |-- TRX_ID: long (nullable = true)



In [17]:
from pyspark.sql.functions import col,when
#df.withColumn('c1', when(df.c1.isNotNull(), 1))
retail_group = retail.select(['CUSTOMER_ID','PAYMENT_MEAN_ID','TRX_ID',month('HED_FECHATRX').alias('mes_unired'),year('HED_FECHATRX').alias('ano_unired'),'SPEND_AMT','DISCOUNT_AMT'])
retail_group = retail_group.join(pagos_distintos,'PAYMENT_MEAN_ID')
retail_group.show()

+---------------+-----------+-------------------+----------+----------+---------+-----------------------+
|PAYMENT_MEAN_ID|CUSTOMER_ID|             TRX_ID|mes_unired|ano_unired|SPEND_AMT|PAYMENT_MEAN_GROUP_NAME|
+---------------+-----------+-------------------+----------+----------+---------+-----------------------+
|             67|   24045870|1610171748281743008|        10|      2016|      319|   TARJETA CASAS COM...|
|             67|   24045870|1610171748281743008|        10|      2016|      818|   TARJETA CASAS COM...|
|             67|   93634008|1610171942471907003|        10|      2016|    18980|   TARJETA CASAS COM...|
|             67|   26207644|1703041423500735004|         3|      2017|     1116|   TARJETA CASAS COM...|
|             67|   17276728|1703041335130735008|         3|      2017|     2014|   TARJETA CASAS COM...|
|             67|   46978114|1610231655340759009|        10|      2016|      269|   TARJETA CASAS COM...|
|             67|   46978114|16102316553407590

In [18]:
## Reemplazamos por 1 a quienes pagan con unimarc 

retail_group = retail_group.withColumn('PAYMENT_MEAN_ID', when(retail_group["PAYMENT_MEAN_GROUP_NAME"] == 'TARJETA UNIMARC', 1).otherwise(0)) #retail_group['PAYMENT_MEAN_ID']))


#retail_group = retail_group.withColumn('PAYMENT_MEAN_ID', when(retail_group["PAYMENT_MEAN_GROUP_NAME"] == 'DEBITO TBK', 1).otherwise(retail_group['PAYMENT_MEAN_ID']))
#retail_group = retail_group.withColumn('PAYMENT_MEAN_ID', when(retail_group["PAYMENT_MEAN_GROUP_NAME"] == 'TARJETA CASAS COMERCIALES', 1).otherwise(retail_group['PAYMENT_MEAN_ID']))
#retail_group = retail_group.withColumn('PAYMENT_MEAN_ID', when(retail_group["PAYMENT_MEAN_GROUP_NAME"] == 'TARJETA TBK', 1).otherwise(retail_group['PAYMENT_MEAN_ID']))
#retail_group = retail_group.withColumn('PAYMENT_MEAN_ID', when(retail_group["PAYMENT_MEAN_ID"] != 1, 0).otherwise(retail_group['PAYMENT_MEAN_ID']))
#retail_group.show()

In [19]:
retail_group3 = retail_group.select(['CUSTOMER_ID','TRX_ID',"PAYMENT_MEAN_ID"]).distinct()
uso_promedio_tarjeta = retail_group3.groupBy(['CUSTOMER_ID']).agg(F.avg('PAYMENT_MEAN_ID')).withColumnRenamed("avg(PAYMENT_MEAN_ID)", "avg_uso_tarjeta").na.drop(how="any")
uso_promedio_tarjeta.show()

+-----------+--------------------+
|CUSTOMER_ID|     avg_uso_tarjeta|
+-----------+--------------------+
|   73631284|                 0.0|
|   87665726|                 0.0|
|   47838114|                 0.0|
|    3104994|                 0.0|
|   77901212|                 0.0|
|   96646272|   0.603448275862069|
|   92385816|                 0.0|
|   11346344|                 0.0|
|   35930268|                 0.0|
|   42272370|                 0.0|
|   88600896|                 0.0|
|   52724058|                 0.0|
|   20174546|                 0.0|
|   55839462|                 0.0|
|   28877086|0.018867924528301886|
|   52809492|                 0.0|
|   49740398|                 0.0|
|   28035302|                 0.0|
|   74412344|                 0.0|
|   17541300|                 0.0|
+-----------+--------------------+
only showing top 20 rows



In [68]:
#corresponde a las variaciones de las persona en sus gastos
retail_gasto = retail.select(['CUSTOMER_ID', 'PAYMENT_MEAN_ID' ,month('HED_FECHATRX').alias('mes_unired'),year('HED_FECHATRX').alias('ano_unired'),'SPEND_AMT', 'DISCOUNT_AMT', 'TRX_ID'])
retail_gasto = retail_gasto.groupBy([retail_gasto['CUSTOMER_ID'], retail_gasto['PAYMENT_MEAN_ID'],retail_gasto['ano_unired'],retail_gasto['mes_unired']]).agg({'SPEND_AMT':'sum','DISCOUNT_AMT':'sum'}).withColumnRenamed("sum(SPEND_AMT)", "pago_retail_mensual").withColumnRenamed("sum(DISCOUNT_AMT)", "DISCOUNT_AMT")
retail_gasto = retail_gasto.groupBy(['CUSTOMER_ID', 'PAYMENT_MEAN_ID']).agg(F.avg('pago_retail_mensual'), F.max('pago_retail_mensual'), F.stddev('pago_retail_mensual'), F.sum('pago_retail_mensual'), F.sum('DISCOUNT_AMT')).withColumnRenamed("avg(pago_retail_mensual)", "avg_pago_retail_mensual").withColumnRenamed("max(pago_retail_mensual)", "max_pago_retail_mensual").withColumnRenamed("stddev_samp(pago_retail_mensual)", "std_pago_retail_mensual" ).withColumnRenamed("sum(pago_retail_mensual)", "SPEND_AMT").withColumnRenamed("sum(DISCOUNT_AMT)", "DISCOUNT_AMT").na.drop(how="any")
retail_gasto.show()

+-----------+---------------+-----------------------+-----------------------+-----------------------+---------+------------+
|CUSTOMER_ID|PAYMENT_MEAN_ID|avg_pago_retail_mensual|max_pago_retail_mensual|std_pago_retail_mensual|SPEND_AMT|DISCOUNT_AMT|
+-----------+---------------+-----------------------+-----------------------+-----------------------+---------+------------+
|   46272988|             63|               10048.75|                  14166|      4505.739108810747|    40195|         907|
|   66151768|             63|      31287.14285714286|                  45349|     13058.623273895448|   219010|       17213|
|   67267658|             11|     15019.555555555555|                  45937|      14303.13304936292|   135176|        6684|
|    5596816|             63|                13503.5|                  18708|     4454.8926287098475|    54014|         897|
|   70057402|             11|                25264.5|                  52060|      19163.20256898622|   151587|        5239|


In [69]:
retail_gasto = retail_gasto.withColumn('PAYMENT_MEAN_ID',when( (retail_gasto['PAYMENT_MEAN_ID'] == 35) | 
                                                          (retail_gasto['PAYMENT_MEAN_ID'] == 68) | 
                                                          (retail_gasto['PAYMENT_MEAN_ID'] == 69) | 
                                                          (retail_gasto['PAYMENT_MEAN_ID'] == 78),1).otherwise(0))

## Unimos las tablas

In [70]:
unir = pagos_uni.join(unired1,'CUSTOMER_ID')
#unir.show()
unir = unir.join(uso_promedio_tarjeta,'CUSTOMER_ID')
#unir.show()
unir = unir.join(retail_gasto,'CUSTOMER_ID')

In [71]:
aux1 = resp.select(['CUSTOMER_ID','GENDER','EDAD','MOBILE_CONTACTABILITY','EMAIL_CONTACTABILITY','GSE','CIVIL_STATUS', 'bloqueo'])

unir = unir.join(aux1, 'CUSTOMER_ID')

In [72]:
muestra = unir.toPandas()

In [73]:
muestra.head()

,CUSTOMER_ID,std_pago_unired,max_pago_unired,avg_pago_unired,pago_unired_mensual,avg_dia_pago_unired,max_dia_pago_unired,min_dia_pago_unired,avg_uso_tarjeta,PAYMENT_MEAN_ID,...,std_pago_retail_mensual,SPEND_AMT,DISCOUNT_AMT,GENDER,EDAD,MOBILE_CONTACTABILITY,EMAIL_CONTACTABILITY,GSE,CIVIL_STATUS,bloqueo
0,9097662,8457.780301,28450,11578.571429,81050,4.371121,7.328028,1.414214,0.0,0,...,20455.093989,655379,56912,MASCULINO,36,1,0,D,SOLTERO,1
1,9097662,8457.780301,28450,11578.571429,81050,4.371121,7.328028,1.414214,0.0,0,...,20455.093989,655379,56912,MASCULINO,36,1,0,D,SOLTERO,1
2,9097662,8457.780301,28450,11578.571429,81050,4.371121,7.328028,1.414214,0.0,0,...,20455.093989,655379,56912,MASCULINO,36,1,0,D,SOLTERO,1
3,9097662,8457.780301,28450,11578.571429,81050,4.371121,7.328028,1.414214,0.0,0,...,20455.093989,655379,56912,MASCULINO,36,1,0,D,SOLTERO,1
4,9097662,8457.780301,28450,11578.571429,81050,4.371121,7.328028,1.414214,0.0,0,...,20455.093989,655379,56912,MASCULINO,36,1,0,D,SOLTERO,1


In [74]:
muestra.to_csv('muestra2desbalanceada.csv', sep='\t', encoding='utf-8', index=False, header=True)

In [75]:
import pandas as pd
num = len(muestra[muestra["bloqueo"] == 1])
print(num)

nbloq = muestra[muestra["bloqueo"] == 0].sample(n = num)
sbloq = muestra[muestra["bloqueo"] == 1].sample(n = num)

muestra1 = pd.concat([nbloq, sbloq])

for i in range(20):
    muestra1 = muestra1.sample(frac = 1).reset_index(drop = True)

10611


In [76]:
muestra1.to_csv('bloqueo2balanceado.csv', sep='\t', encoding='utf-8', index=False, header=True)

In [77]:
num = len(muestra[muestra["PAYMENT_MEAN_ID"] == 1])
print(num)

nbloq = muestra[muestra["PAYMENT_MEAN_ID"] == 0].sample(n = num)
sbloq = muestra[muestra["PAYMENT_MEAN_ID"] == 1].sample(n = num)

muestra2 = pd.concat([nbloq, sbloq])

for i in range(20):
    muestra2 = muestra2.sample(frac = 1).reset_index(drop = True)

4314


In [78]:
muestra2.to_csv('unicard2balanceado.csv', sep='\t', encoding='utf-8', index=False, header=True)

In [1]:
!pwd

/gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/work


In [3]:
!mkdir /gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/data2

mkdir: cannot create directory ‘/gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/data2’: File exists


In [4]:
!cp bloqueo2balanceado.csv /gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/data2
!cp unicard2balanceado.csv /gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/data2
!cp muestra2desbalanceada.csv /gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/data2

In [5]:
!ls /gpfs/fs01/user/s57f-73edb3521eee1d-79e3f294add4/notebook/data2

bloqueo2balanceado.csv	muestra2desbalanceada.csv  unicard2balanceado.csv
